In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import warnings

In [2]:
warnings.filterwarnings("ignore")
rcParams['figure.figsize'] = 12, 8

In [3]:
def show_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [4]:
%%time
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
submission = pd.read_csv('input/sample_submission.csv')

CPU times: user 61.9 ms, sys: 17.1 ms, total: 78.9 ms
Wall time: 77.5 ms


In [5]:
train.shape, test.shape, submission.shape

((21000, 25), (9000, 24), (9000, 2))

In [6]:
show_all(train.head())

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,25150,360000,2,2,2,25,-1,-1,-1,-1,-1,-1,20952,12590,3479,3435,8870,2020,12590,3479,3446,8870,2020,27043,0
1,13470,240000,2,1,2,46,2,2,-2,-1,0,-1,456,0,0,2240,1681,2267,0,0,2240,0,2267,3074,0
2,3092,320000,2,2,1,41,0,0,0,0,0,0,49846,53811,38717,35409,35036,35939,5000,2000,2000,3000,3000,1500,0
3,13973,50000,2,2,2,24,-1,-1,-2,-2,-2,-2,2675,0,0,0,0,0,0,0,0,0,0,0,1
4,10567,80000,1,3,1,52,-1,0,0,0,0,0,35861,18521,19385,20561,21261,21457,1500,1500,1500,1028,876,1449,0


In [7]:
show_all(test.head())

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,10178,60000,2,2,1,30,0,0,0,0,0,0,36082,37127,38117,38874,38667,39544,1628,1618,1375,1403,1520,1205
1,5304,200000,2,1,2,29,0,0,0,0,2,0,61099,64160,76593,93988,50292,10510,4160,14593,20150,0,5739,12050
2,5187,230000,1,1,2,39,0,0,0,0,-2,-2,38395,39943,20650,0,0,0,2500,2000,0,0,0,0
3,14495,150000,1,1,2,40,0,0,0,0,0,0,152175,148872,145978,139791,142745,146872,5800,5245,5200,5200,6500,7200
4,20444,140000,1,2,2,27,0,0,0,0,0,0,80210,67746,76230,77954,78953,81848,10000,10000,3393,3260,5000,3000


In [8]:
show_all(submission.head())

,ID,default_payment_next_month
0,10178,0
1,5304,0
2,5187,0
3,14495,0
4,20444,0


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 25 columns):
ID                            21000 non-null int64
LIMIT_BAL                     21000 non-null int64
SEX                           21000 non-null int64
EDUCATION                     21000 non-null int64
MARRIAGE                      21000 non-null int64
AGE                           21000 non-null int64
PAY_0                         21000 non-null int64
PAY_2                         21000 non-null int64
PAY_3                         21000 non-null int64
PAY_4                         21000 non-null int64
PAY_5                         21000 non-null int64
PAY_6                         21000 non-null int64
BILL_AMT1                     21000 non-null int64
BILL_AMT2                     21000 non-null int64
BILL_AMT3                     21000 non-null int64
BILL_AMT4                     21000 non-null int64
BILL_AMT5                     21000 non-null int64
BILL_AMT6               

In [10]:
show_all(train.iloc[:,1:].describe())

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
count,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.00000,21000.000000,21000.000000,21000.00000,21000.000000,21000.000000,21000.000000,21000.000000,2.100000e+04,21000.000000,21000.000000,21000.000000,21000.000000,2.100000e+04,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000
mean,167214.746667,1.607571,1.854190,1.551714,35.461619,-0.01119,-0.127238,-0.164857,-0.21819,-0.260952,-0.288667,51501.542381,49463.502667,4.723258e+04,43387.372476,40398.551095,38931.194000,5686.349333,5.923003e+03,5202.325333,4793.172000,4797.012952,5211.736762,0.221190
std,128965.188482,0.488303,0.791628,0.521176,9.206628,1.12321,1.198957,1.198624,1.17221,1.141454,1.151592,73453.641859,70866.586004,6.953988e+04,64081.073110,60396.811177,59196.499234,16868.075695,2.390953e+04,17006.416467,15467.403159,15270.031988,17698.795697,0.415058
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.00000,-2.000000,-2.000000,-2.00000,-2.000000,-2.000000,-14386.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-209051.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.00000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,3564.750000,3000.000000,2.686250e+03,2332.000000,1759.000000,1242.750000,998.250000,8.360000e+02,390.000000,284.000000,241.000000,102.000000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,22578.000000,21550.000000,2.024200e+04,19158.500000,18266.500000,17203.500000,2100.000000,2.011000e+03,1811.500000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,67876.250000,64918.250000,6.082675e+04,55376.750000,50517.250000,49463.000000,5023.250000,5.000000e+03,4500.000000,4002.250000,4051.000000,4000.000000,0.000000
max,800000.000000,2.000000,6.000000,3.000000,75.000000,8.00000,8.000000,8.000000,8.00000,8.000000,8.000000,746814.000000,743970.000000,1.664089e+06,706864.000000,587067.000000,699944.000000,873552.000000,1.684259e+06,889043.000000,621000.000000,417990.000000,528666.000000,1.000000


In [11]:
for df in (train, test):
    missing_stats = df.isnull().sum()
    print(missing_stats[missing_stats != 0])

Series([], dtype: int64)
Series([], dtype: int64)


In [12]:
test['default_payment_next_month'] = -1

In [13]:
full_data = train.append(test)
full_data.shape

(30000, 25)

In [14]:
full_data = full_data.rename(columns={'PAY_0' : 'PAY_1'})

## Exploratory Analysis

In [15]:
id_col = ['ID']
cat_cols = ['SEX', 'EDUCATION', 'MARRIAGE']
PAY_cols = ['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
PMT_cols = list(filter(lambda x: 'PAY_AMT' in x, train.columns))
BILL_AMT_cols = list(filter(lambda x: 'BILL_AMT' in x, train.columns))
target_col = 'default_payment_next_month'

In [16]:
for col in full_data.columns:
    print(f"{col}: {full_data[col].nunique()}")

ID: 30000
LIMIT_BAL: 81
SEX: 2
EDUCATION: 7
MARRIAGE: 4
AGE: 56
PAY_1: 11
PAY_2: 11
PAY_3: 11
PAY_4: 11
PAY_5: 10
PAY_6: 10
BILL_AMT1: 22723
BILL_AMT2: 22346
BILL_AMT3: 22026
BILL_AMT4: 21548
BILL_AMT5: 21010
BILL_AMT6: 20604
PAY_AMT1: 7943
PAY_AMT2: 7899
PAY_AMT3: 7518
PAY_AMT4: 6937
PAY_AMT5: 6897
PAY_AMT6: 6939
default_payment_next_month: 3


In [17]:
train[target_col].value_counts(normalize=True)

0    0.77881
1    0.22119
Name: default_payment_next_month, dtype: float64

In [18]:
for col in cat_cols:
    print(f"{col}:\n{train[col].value_counts()}\n")

SEX:
2    12759
1     8241
Name: SEX, dtype: int64

EDUCATION:
2    9789
1    7415
3    3467
5     200
4      85
6      34
0      10
Name: EDUCATION, dtype: int64

MARRIAGE:
2    11184
1     9561
3      219
0       36
Name: MARRIAGE, dtype: int64



## Feature Engineering

In [19]:
show_all(full_data.head(5))

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,25150,360000,2,2,2,25,-1,-1,-1,-1,-1,-1,20952,12590,3479,3435,8870,2020,12590,3479,3446,8870,2020,27043,0
1,13470,240000,2,1,2,46,2,2,-2,-1,0,-1,456,0,0,2240,1681,2267,0,0,2240,0,2267,3074,0
2,3092,320000,2,2,1,41,0,0,0,0,0,0,49846,53811,38717,35409,35036,35939,5000,2000,2000,3000,3000,1500,0
3,13973,50000,2,2,2,24,-1,-1,-2,-2,-2,-2,2675,0,0,0,0,0,0,0,0,0,0,0,1
4,10567,80000,1,3,1,52,-1,0,0,0,0,0,35861,18521,19385,20561,21261,21457,1500,1500,1500,1028,876,1449,0


## Creating New Features

In [20]:
full_data['EDUCATION'] = np.where((full_data['EDUCATION'] > 4) | (full_data['EDUCATION'] == 0), 4, full_data['EDUCATION'])

In [21]:
full_data['MARRIAGE'] = np.where(full_data['MARRIAGE'] == 0, 3, full_data['MARRIAGE'])

In [22]:
full_data['Repayment_stats_sum'] = full_data[PAY_cols].sum(axis=1)

In [23]:
full_data['Mean_Bill_Amts'] = full_data[BILL_AMT_cols].mean(axis=1)
full_data['Mean_Pay_Amts'] = full_data[PMT_cols].mean(axis=1)

In [24]:
full_data['Min_Bill_Amts'] = full_data[BILL_AMT_cols].min(axis=1)
full_data['Min_Pay_Amts'] = full_data[PMT_cols].min(axis=1)

In [25]:
full_data['Max_Bill_Amts'] = full_data[BILL_AMT_cols].max(axis=1)
full_data['Max_Pay_Amts'] = full_data[PMT_cols].max(axis=1)

In [26]:
full_data['Min_Max_Bill_Amts_Diff'] = full_data['Max_Bill_Amts'] - full_data['Min_Bill_Amts']
full_data['Min_Max_Pay_Amts_Diff'] = full_data['Max_Pay_Amts'] - full_data['Min_Pay_Amts']

In [27]:
full_data['Prev_5_BILL_AMTS'] = full_data['BILL_AMT6'] + full_data['BILL_AMT5'] + full_data['BILL_AMT4'] + full_data['BILL_AMT3'] + full_data['BILL_AMT2']
full_data['Prev_5_PAY_AMTS'] = full_data['PAY_AMT5'] + full_data['PAY_AMT4'] + full_data['PAY_AMT3'] + full_data['PAY_AMT2'] + full_data['PAY_AMT1']


In [28]:
full_data['BILL_PAY_Diff'] = full_data['Prev_5_BILL_AMTS'] - full_data['Prev_5_PAY_AMTS']

In [29]:
full_data['BILL_PAY_RATIO'] = full_data['Prev_5_PAY_AMTS']/full_data['Prev_5_BILL_AMTS']
full_data['Next_Expected_PAY_AMT'] = full_data['BILL_PAY_RATIO'] * full_data['BILL_AMT1']
full_data['Next_Bill_Expected_Diff'] = full_data['BILL_AMT1'] - full_data['Next_Expected_PAY_AMT']

In [30]:
for col in enumerate(BILL_AMT_cols):
    full_data[f'Pct_bal_used_{col[0]+1}'] = full_data[col[1]]/full_data['LIMIT_BAL']

In [31]:
show_all(full_data.head())

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month,Repayment_stats_sum,Mean_Bill_Amts,Mean_Pay_Amts,Min_Bill_Amts,Min_Pay_Amts,Max_Bill_Amts,Max_Pay_Amts,Min_Max_Bill_Amts_Diff,Min_Max_Pay_Amts_Diff,Prev_5_BILL_AMTS,Prev_5_PAY_AMTS,BILL_PAY_Diff,BILL_PAY_RATIO,Next_Expected_PAY_AMT,Next_Bill_Expected_Diff,Pct_bal_used_1,Pct_bal_used_2,Pct_bal_used_3,Pct_bal_used_4,Pct_bal_used_5,Pct_bal_used_6
0,25150,360000,2,2,2,25,-1,-1,-1,-1,-1,-1,20952,12590,3479,3435,8870,2020,12590,3479,3446,8870,2020,27043,0,-6,8557.666667,9574.666667,2020,2020,20952,27043,18932,25023,30394,30405,-11,1.000362,20959.582812,-7.582812,0.058200,0.034972,0.009664,0.009542,0.024639,0.005611
1,13470,240000,2,1,2,46,2,2,-2,-1,0,-1,456,0,0,2240,1681,2267,0,0,2240,0,2267,3074,0,0,1107.333333,1263.500000,0,0,2267,3074,2267,3074,6188,4507,1681,0.728345,332.125404,123.874596,0.001900,0.000000,0.000000,0.009333,0.007004,0.009446
2,3092,320000,2,2,1,41,0,0,0,0,0,0,49846,53811,38717,35409,35036,35939,5000,2000,2000,3000,3000,1500,0,0,41459.666667,2750.000000,35036,1500,53811,5000,18775,3500,198912,15000,183912,0.075410,3758.898407,46087.101593,0.155769,0.168159,0.120991,0.110653,0.109488,0.112309
3,13973,50000,2,2,2,24,-1,-1,-2,-2,-2,-2,2675,0,0,0,0,0,0,0,0,0,0,0,1,-10,445.833333,0.000000,0,0,2675,0,2675,0,0,0,0,NaN,NaN,NaN,0.053500,0.000000,0.000000,0.000000,0.000000,0.000000
4,10567,80000,1,3,1,52,-1,0,0,0,0,0,35861,18521,19385,20561,21261,21457,1500,1500,1500,1028,876,1449,0,-1,22841.000000,1308.833333,18521,876,35861,1500,17340,624,101185,6404,94781,0.063290,2269.643168,33591.356832,0.448263,0.231513,0.242312,0.257012,0.265763,0.268213


## Modeling

In [32]:
import sys
sys.path.append('ml_modules/')

In [33]:
from custom_estimator import Estimator
from encoding import FreqeuncyEncoding
from lightgbm import LGBMClassifier
import category_encoders as enc
from sklearn.metrics import roc_auc_score
from sklearn import model_selection

Using TensorFlow backend.


In [34]:
train_data = full_data[full_data[target_col] != -1]
test_data = full_data[full_data[target_col] == -1]

In [35]:
categorical_cols = cat_cols

In [36]:
fE = FreqeuncyEncoding(categorical_columns=categorical_cols, return_df=True)
train_data = fE.fit_transform(train_data)
test_data = fE.transform(test_data)

In [37]:
# Target Encoding
X = train_data[categorical_cols]
y = train_data[target_col].values

enc_train = np.zeros(X.shape)
smoothing = 0.3

Folds = model_selection.KFold(n_splits=5, random_state=2020, shuffle=True)

for train_idx, valid_idx in Folds.split(X):
    encoder = enc.TargetEncoder(cols=categorical_cols, smoothing=smoothing)

    encoder.fit(X.iloc[train_idx], y[train_idx])
    enc_train[valid_idx, :] = encoder.transform(X.iloc[valid_idx], y[valid_idx])

encoder.fit(X, y)
enc_test = encoder.transform(test_data[categorical_cols]).values

for idx, col in enumerate(categorical_cols):
    col = 'tE_' + col 
    train_data[col] = enc_train[:,idx]
    test_data[col] = enc_test[:, idx]

In [38]:
drop_cols = id_col + [target_col]
y = train_data[target_col]

train_data.drop(columns=drop_cols, axis=1, inplace=True)
test_data.drop(columns=drop_cols, axis=1, inplace=True)

In [39]:
train_data = train_data.replace({np.inf: np.nan, -np.inf: np.nan}).fillna(-999)
test_data = test_data.replace({np.inf: np.nan, -np.inf: np.nan}).fillna(-999)

In [40]:
for df in (train_data, test_data):
    missing_stats = df.isnull().mean()
    print(missing_stats[missing_stats != 0])

Series([], dtype: float64)
Series([], dtype: float64)


In [41]:
train_data.shape, test_data.shape

((21000, 47), (9000, 47))

In [42]:
show_all(train_data.head())

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Repayment_stats_sum,Mean_Bill_Amts,Mean_Pay_Amts,Min_Bill_Amts,Min_Pay_Amts,Max_Bill_Amts,Max_Pay_Amts,Min_Max_Bill_Amts_Diff,Min_Max_Pay_Amts_Diff,Prev_5_BILL_AMTS,Prev_5_PAY_AMTS,BILL_PAY_Diff,BILL_PAY_RATIO,Next_Expected_PAY_AMT,Next_Bill_Expected_Diff,Pct_bal_used_1,Pct_bal_used_2,Pct_bal_used_3,Pct_bal_used_4,Pct_bal_used_5,Pct_bal_used_6,tE_SEX,tE_EDUCATION,tE_MARRIAGE
0,360000,0.607571,0.466143,0.532571,25,-1,-1,-1,-1,-1,-1,20952,12590,3479,3435,8870,2020,12590,3479,3446,8870,2020,27043,-6,8557.666667,9574.666667,2020,2020,20952,27043,18932,25023,30394,30405,-11,1.000362,20959.582812,-7.582812,0.058200,0.034972,0.009664,0.009542,0.024639,0.005611,0.207178,0.241706,0.212179
1,240000,0.607571,0.353095,0.532571,46,2,2,-2,-1,0,-1,456,0,0,2240,1681,2267,0,0,2240,0,2267,3074,0,1107.333333,1263.500000,0,0,2267,3074,2267,3074,6188,4507,1681,0.728345,332.125404,123.874596,0.001900,0.000000,0.000000,0.009333,0.007004,0.009446,0.209086,0.196260,0.214214
2,320000,0.607571,0.466143,0.455286,41,0,0,0,0,0,0,49846,53811,38717,35409,35036,35939,5000,2000,2000,3000,3000,1500,0,41459.666667,2750.000000,35036,1500,53811,5000,18775,3500,198912,15000,183912,0.075410,3758.898407,46087.101593,0.155769,0.168159,0.120991,0.110653,0.109488,0.112309,0.211576,0.240331,0.234020
3,50000,0.607571,0.466143,0.532571,24,-1,-1,-2,-2,-2,-2,2675,0,0,0,0,0,0,0,0,0,0,0,-10,445.833333,0.000000,0,0,2675,0,2675,0,0,0,0,-999.000000,-999.000000,-999.000000,0.053500,0.000000,0.000000,0.000000,0.000000,0.000000,0.209086,0.237834,0.214214
4,80000,0.392429,0.165095,0.455286,52,-1,0,0,0,0,0,35861,18521,19385,20561,21261,21457,1500,1500,1500,1028,876,1449,-1,22841.000000,1308.833333,18521,876,35861,1500,17340,624,101185,6404,94781,0.063290,2269.643168,33591.356832,0.448263,0.231513,0.242312,0.257012,0.265763,0.268213,0.240414,0.248830,0.234020


In [43]:
lgb_params = {
    'n_estimators': 20000, 
    'learning_rate': 0.01,
    'boosting_type': 'gbdt', 
    'colsample_bytree': 0.80, 
    'min_child_weight': 9.0, 
    'num_leaves': 64, 
    'objective': 'binary', 
    'subsample': 0.80, 
    'subsample_freq': 5,
    'metric': 'custom'
#     'n_jobs': -1
    }
from sklearn.svm import SVC

lgb_100 = Estimator( LGBMClassifier(**lgb_params)
                     , random_state=100
                     , eval_metric='AUC'
                     , scoring_metric=roc_auc_score
                     , early_stopping_rounds=200
                     , verbose=200
                    )

In [44]:
lgb_oof_100 = lgb_100.fit_transform(train_data, y.values)

Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.865141	valid's auc: 0.784604
Early stopping, best iteration is:
[55]	train's auc: 0.835896	valid's auc: 0.785259
Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.868249	valid's auc: 0.784841
[400]	train's auc: 0.905254	valid's auc: 0.784133
Early stopping, best iteration is:
[260]	train's auc: 0.87959	valid's auc: 0.78565
Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.867496	valid's auc: 0.78576
Early stopping, best iteration is:
[48]	train's auc: 0.834985	valid's auc: 0.78655
Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.864361	valid's auc: 0.787624
Early stopping, best iteration is:
[191]	train's auc: 0.862594	valid's auc: 0.787748
Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.862885	valid's auc: 0.792758
[400]	train's auc: 0.899138	valid's auc: 0.793795
Early 

In [45]:
lgb_200 = Estimator(LGBMClassifier(**lgb_params)
                     , random_state=200
                     , eval_metric='AUC'
                     , scoring_metric=roc_auc_score
                     , early_stopping_rounds=200
                     , verbose=200
                    )

In [46]:
lgb_oof_200 = lgb_200.fit_transform(train_data, y.values)

Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.865527	valid's auc: 0.790605
Early stopping, best iteration is:
[196]	train's auc: 0.864527	valid's auc: 0.790716
Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.865483	valid's auc: 0.787669
[400]	train's auc: 0.901774	valid's auc: 0.789301
Early stopping, best iteration is:
[379]	train's auc: 0.898311	valid's auc: 0.789438
Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.864254	valid's auc: 0.783405
Early stopping, best iteration is:
[100]	train's auc: 0.844049	valid's auc: 0.784326
Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.865086	valid's auc: 0.781306
[400]	train's auc: 0.901568	valid's auc: 0.781185
Early stopping, best iteration is:
[267]	train's auc: 0.878418	valid's auc: 0.782486
Training until validation scores don't improve for 200 rounds
[200]	train's auc: 0.866369	valid's auc: 0.792775


In [47]:
lgb_100.avg_cv_score, lgb_200.avg_cv_score

(0.7879036146005656, 0.7882085746187835)

In [48]:
lgb_100.feature_importances(columns=train_data.columns).head(10)

,column,feature_importance,rank
36,Next_Expected_PAY_AMT,0.045950,1
39,Pct_bal_used_2,0.042133,2
4,AGE,0.041117,3
0,LIMIT_BAL,0.039089,4
30,Min_Max_Bill_Amts_Diff,0.038637,5
23,Repayment_stats_sum,0.038346,6
35,BILL_PAY_RATIO,0.038146,7
18,PAY_AMT2,0.035165,8
19,PAY_AMT3,0.032208,9
38,Pct_bal_used_1,0.031226,10


In [49]:
lgb_pred_100 = lgb_100.transform(test_data)
lgb_pred_200 = lgb_200.transform(test_data)

In [50]:
final_sub = submission.copy()

In [51]:
final_sub[target_col] = np.mean([lgb_pred_100, lgb_pred_200], axis=0)

In [52]:
final_sub.head()

,ID,default_payment_next_month
0,10178,0.190545
1,5304,0.131177
2,5187,0.116867
3,14495,0.122319
4,20444,0.090155


In [53]:
file_timestamp = datetime.now().strftime('%m%d%Y_%H%M')
model_name='LGBM'

In [54]:
final_sub.to_csv(f"output/{model_name}_{file_timestamp}.csv", index=False)